In [17]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np


from sklearn.decomposition import PCA
import torch


In [8]:
with open("pkls/data1_inference_math1.pkl", "rb") as fp:
    data_list = pickle.load(fp)

In [9]:
def gather_norm_acts(data):
    list_norm_act = []
    # For each dataset element (words)
    for word_id in range(len(data)):
        list_norm_act.append([])
        # For the subset of selected layers in the dictionaray
        for layer_nb in data[word_id]["hook"].keys():
            list_norm_act[word_id].append(data[word_id]["hook"][layer_nb]["normalized"])
    return list_norm_act

# Selecting last element of the generation - AKA the Yes generation or No generation
data_acts = gather_norm_acts(data_list)

print("Just a list of layer size with each embedding size vector")
print(len(data_acts))          # Size of word dataset
print(len(data_acts[0]))       # Size of layer (subset)
print(len(data_acts[0][0]))    # Size of output
print(len(data_acts[0][0][0])) # Size of enbedding

Just a list of layer size with each embedding size vector
300
32
1
4096


In [10]:
def flat_data(data, words, layers, tokens):
    return [data[w][l][t] for w in words for l in layers for t in tokens]

In [11]:
color_selction = ["#ef6c00", "#eeff41", "#009668", "#ce93d8" , "#a1e8d9", "#b3a77d", "#6a634a"]

def simple_label2color(label):
    # labels are 4 choices
    color = color_selction[label]
    return color

def bidim_label2color(label):
    # labels are 2*2*2 choices
    color = color_selction[label[1] + 2*label[2]]
    # color = mcolors.CSS4_COLORS[color_selction[label[1] + 2*label[2]]]
    return color

In [59]:
label_colors = \
    [simple_label2color(0) for _ in range(len(data_acts))]

layer_pca = []
layer_id = []
for layer_nb in range(32):
    # Gather the list correctly
    data_states = torch.stack(flat_data(data_acts, range(len(data_acts)), [layer_nb], [-1])).type(torch.float64).cpu().numpy()
    # Train on one problem
    pca = PCA(n_components=2)
    x = pca.fit_transform(data_states)
    layer_pca.extend(x)
    a = np.empty(len(x))
    a.fill(layer_nb)
    layer_id.extend(a)
layer_pca = np.array(layer_pca)
layer_id = np.array(layer_id)

y = np.array([data_list[i]["label"]+ [data_list[i]["result"]] for i in range(len(data_list))])
print(y.shape)
y_layered = np.tile(y, (32, 1))

inputs = np.array([data_list[i]["input"] + data_list[i]["output"] for i in range(len(data_list))])
print(inputs.shape)
inputs_layered = np.tile(inputs, 32)


print(y_layered.shape)
print(layer_pca.shape)
print(layer_id.shape)
print(inputs_layered.shape)

df = pd.DataFrame(zip(layer_pca[:, 0], layer_pca[:, 1], y_layered[:, 0], y_layered[:, 1], y_layered[:, 2], y_layered[:, 3], inputs_layered, layer_id), 
                  columns=['pca1', 'pca2', 'first digit', 'second digit', 'operation type', 'result', "input/output", "layer_id"])


df["result"] = df["result"] + 10

df = df[df["operation type"] == 2]
df = df[df["layer_id"]%10 == 0]

scatter = px.scatter(df, x='pca1', y='pca2', 
                     color="first digit",
                     size='result', 
                     symbol= "operation type",
                     facet_row="layer_id",
                     hover_data=["input/output", "layer_id"])

scatter.update_layout(
    autosize=True,
    width=1400,
    height=1600,)

scatter.show()

(300, 4)
(300,)
(9600, 4)
(9600, 2)
(9600,)
(9600,)
